## 复现内容：
### （1）python 实现基于贝叶斯的 IMDB 文本情感分类算法。
### （2）掌握文本类数据特征处理方式。
### （3）掌握 TF_IDF 的基本思想，并使用 TD_IDF 实现新闻话题分类预测。

#### 加载库函数

In [1]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'nltk'

In [2]:
# 加载数据集
imdb_data=pd.read_csv('data/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


### 数据分析

In [3]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [4]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [14]:
# 模型训练过程，输出准确率
X=imdb_data['review']
y=imdb_data['sentiment']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.2,
                                                        random_state=6)

40132    This is one of the best episodes of Doctor Who...
31309    it's a very nice movie and i would definitely ...
39687    This film, although not totally bad, should ha...
7977     I found 'Shuttle' an incredibly frustrating fi...
25729    Just as a reminder to anyone just now reading ...
                               ...                        
41295    If you want to see a movie with nudity, sex, d...
4714     I enjoyed this film. The way these mutants loo...
42964    I'm somewhat of a fan of Lynche's work, so I w...
41187    These are one of the movies that don't require...
31626    There are 3 of these movies, all similarly cra...
Name: review, Length: 40000, dtype: object